<a href="https://colab.research.google.com/github/lgiesen/forest_height/blob/main/notebooks/0_generate_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data

In [1]:
from google.colab import drive
drive.mount ('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
# the zipped data is uploaded in the root_path folder
root_path = 'drive/MyDrive/Colab Notebooks/data/'
path_images = f'{root_path}images/'
path_masks = f'{root_path}masks/'
user = "lgiesen"
repo = "forest_height"
!git clone https://github.com/{user}/{repo}.git

fatal: destination path 'forest_height' already exists and is not an empty directory.


In [3]:
%run /content/forest_height/src/generate_data.py

In [4]:
zip_files = [filename for filename in get_files(root_path) if '.zip' in filename]
zip_files

['masks_train.zip', 'images_train.zip', 'masks_02.zip', 'images_02.zip']

In [ ]:
%%time
# unzip data
%cd "drive/MyDrive/Colab Notebooks/data/"
# use -B flag to rename files if there is a file with its name
!for f in *.zip; do unzip -B "$f"; done
%cd ../../../../

/content/drive/MyDrive/Colab Notebooks/data
Archive:  images_02.zip
  inflating: images/image_000.npy    
  inflating: images/image_001.npy    
  inflating: images/image_002.npy    
  inflating: images/image_003.npy    
  inflating: images/image_004.npy    
  inflating: images/image_005.npy    
  inflating: images/image_006.npy    
  inflating: images/image_007.npy    
  inflating: images/image_008.npy    
  inflating: images/image_009.npy    
  inflating: images/image_010.npy    
  inflating: images/image_011.npy    
  inflating: images/image_012.npy    
  inflating: images/image_013.npy    
  inflating: images/image_014.npy    
  inflating: images/image_015.npy    
  inflating: images/image_016.npy    
  inflating: images/image_017.npy    
  inflating: images/image_018.npy    
  inflating: images/image_019.npy    
Archive:  images_train.zip
  inflating: images/image_000.npy    
  inflating: images/image_001.npy    
  inflating: images/image_002.npy    
  inflating: images/image_003.n

In [5]:
%%time
X_train, X_test, y_train, y_test = generate_dataset(zip_files)

CPU times: user 11.9 s, sys: 18.3 s, total: 30.2 s
Wall time: 39 s


In [6]:
# remove drive connection as it is no longer needed
drive.flush_and_unmount()